In [1]:
!pip install -r requirements.txt -q

In [40]:
from easyocr import Reader
from pytube import YouTube

import time
import pandas as pd
import cv2
import whisper
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [3]:
langs = ['ko', 'en']
# joblib.dump(Reader(lang_list=langs, gpu=False), 'easyocr_base_model.pkl')
easyocr_model = Reader(lang_list=langs, gpu=False)

# joblib.dump(whisper.load_model("base"), "whisper_base_model.pkl")
whisper_model = whisper.load_model("base")

In [4]:
url = 'https://www.youtube.com/watch?v=bGcVkNP1tPs&t=2s&ab_channel=1%EB%B6%84%EB%AF%B8%EB%A7%8C'

yt = YouTube(url)
# 비디오
best_video = yt.streams.get_highest_resolution()
# 오디오
audio_ = yt.streams.filter(only_audio=True).first()

In [ ]:
# 자막 리스트 중 첫 번째 자막 선택
caption = yt.captions.all()[0]
# 한국어 가져오기 'ko'와 'a.ko' 존재. 'a.ko'는 자동 생성된 한국어 자막
# 영어도 마찬가지로 'en'과 'a.en' 존재.
# caption = yt.captions.get('a.ko')

# YouTube에서 제공하는 자막 DataFrame으로 저장
caption_df = pd.DataFrame(columns=["start", "end", "text"])
for data in caption.json_captions['events']:
    if 'aAppend' in data:
        continue
    if 'segs' in data:
        start = data['tStartMs'] / 1000.0
        end = start + (data['dDurationMs'] / 1000.0)
        text = []
        for seg in data['segs']:
            text.append(seg['utf8'])
        text = ' '.join(text)
        temp = pd.DataFrame([[start, end, text]], columns=["start", "end", "text"])
        caption_df = pd.concat([caption_df, temp], ignore_index=True)

In [38]:
caption_df

,start,end,text
0,0.000,3.600,자 오늘은 지금 전 국민이 혼란에
1,1.979,5.160,빠진 우회전 관련 최종 정리
2,3.600,6.839,영상인데요 이거 뉴스에서도 정작
3,5.160,8.220,중요한 건 안 알려줘서 더 헷갈리게
4,6.839,10.200,만드는 시간이 없으니 핵심만 빠르게
...,...,...,...
59,108.780,112.200,건데요 확실히 이러면 더 안전해야
60,110.460,115.340,하겠지만 지금보다 훨씬 더 막히는 건
61,112.200,115.340,어쩔 수 없을 것 같습니다
62,115.399,120.619,그래서 더 이상 단속되는 분들이


In [44]:
# easyocr으로 Text 추출
cap = cv2.VideoCapture(best_video.url)
fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = cap.get(cv2.CAP_PROP_FRAME_COUNT) # 프레임 개수
print("fps :", fps)
print("frame_count :", frame_count)
easyocr_results = pd.DataFrame(columns=["bbox", "text", "conf", "time"])
images = []
start_time = time.time()
for _ in tqdm(range(int(frame_count))):
    frame_pos = cap.get(cv2.CAP_PROP_POS_FRAMES) # 현재 프레임
    success, frame = cap.read()
    if not success:
        print('not success frame :', frame_pos)
        print('not success time :', frame_pos / fps)
        print('not success ... break')
        break
    if frame_pos % fps != 0: # 1초에 한 장씩
        continue
    images.append(frame)
    result = pd.DataFrame(easyocr_model.readtext(frame, detail=1), columns=['bbox', 'text', 'conf'])
    result['time'] = frame_pos / fps 
    easyocr_results = pd.concat([easyocr_results, result], ignore_index=True)
    # if count // fps >= 5: # 5초까지
    #     break
end_time = time.time()
print("time :", end_time - start_time)

fps : 30.0
frame_count : 3594.0


 83%|████████▎ | 2998/3594 [26:18<05:13,  1.90it/s]

not success frame : 2998.0
not success time : 99.93333333333334
not success ... break
time : 1578.4889252185822


In [45]:
easyocr_results

,bbox,text,conf,time
0,"[[497, 254], [787, 254], [787, 340], [497, 340]]",자 오늘은,0.999160,0.0
1,"[[253, 252], [1027, 252], [1027, 343], [253, 3...",지금 전국민이 훈린에 빠진,0.502869,1.0
2,"[[463, 251], [816, 251], [816, 342], [463, 342]]",우회전 관련,0.731145,2.0
3,"[[326, 252], [953, 252], [953, 341], [326, 341]]",(최종정리 영상 인데요,0.747477,3.0
4,"[[268, 204], [342, 204], [342, 236], [268, 236]]",막가각,0.999575,4.0
...,...,...,...,...
144,"[[26, 96], [512, 96], [512, 200], [26, 200]]",관다리 보다늄,0.087057,95.0
145,"[[23, 95], [993, 95], [993, 203], [23, 203]]",돼리에세다 일시정지하고선,0.425776,96.0
146,"[[25, 95], [905, 95], [905, 203], [25, 203]]",라느데 맞다라고 보고서움,0.121264,97.0
147,"[[28, 96], [904, 96], [904, 200], [28, 200]]",가능게 맞다라고 보고서움,0.134659,98.0


In [55]:
start_time = time.time()
audio_all = whisper.load_audio(audio_.url) # load audio
result = whisper_model.transcribe(audio_all)
end_time = time.time() 
print("time :", end_time - start_time)

time : 58.326634883880615


In [48]:
whisper_result = pd.DataFrame(columns=["start", "end", "text"])
for seg in result['segments']:
    start, end, text = seg['start'], seg['end'], seg['text']
    print(f"[ {start:>6.2f} ~ {end:>6.2f} ] {text}")
    temp = pd.DataFrame([[start, end, text]], columns=["start", "end", "text"])
    whisper_result = pd.concat([whisper_result, temp], ignore_index=True)

[   0.00 ~   4.00 ]  자 오늘은 지금 전국민이 혼란에 빠진 우회전 관련 최종정리 영상인데요.
[   4.00 ~   7.04 ]  이거 뉴스에서도 정작 중요한 건 안 알려져서 거의 깔리게 만드는
[   7.04 ~   8.60 ]  시간엽수니 핵심한 빠르게가 부족.
[   8.60 ~  12.68 ]  일단 첫 번째 사실 지금 단속되고 있는 차량 90%는 다 이것 때문인데요.
[  12.68 ~  15.48 ]  바로 도대체 어디서 일시 정지를 해야 되냐는 겁니다.
[  15.48 ~  21.08 ]  결론부터 말씀드리면 대부분 여기 우회전에서 만나는 보행자 신호등에서 멈춰야 되는 걸로 알고 있는데요.
[  21.08 ~  21.68 ]  아닙니다.
[  21.68 ~  26.44 ]  정답은 바로 여기 우회전하기 전에 만나는 횡단보도에 있는 정지선에서 일시 정지해야 돼요.
[  26.44 ~  28.64 ]  확실하게 경찰청 공식 입장을 들어보죠.
[  28.64 ~  33.44 ]  앞에 운영이 진행하고자 하는 방향에 적식 적신호가 들어왔을 때는
[  33.44 ~  36.00 ]  처음 만나는 횡단보도 앞에 정지선이 있거든요.
[  36.00 ~  42.56 ]  거기에서 일단 정지해서 자후에 부행자가 있는지 없는지 확인하고 그 다음에 진행을 해야 되는 상황이고요.
[  42.56 ~  47.32 ]  네, 한마디로 전방 신호등이 빨간 부릴때 바로 앞 정지선에 맞춰서 일시 정지를 해야 된다는 겁니다.
[  47.32 ~  50.52 ]  그리고 녹색 부릴때는 지금까지와 똑같이 서행에서 우회전을 하던
[  50.52 ~  55.36 ]  부행자 유무에 따라서 천천히 지나갈 수도 있고 다시 한번 멈췄다가 가야 될 수도 있는 거죠.
[  55.36 ~  56.08 ]  바로 두 번째.
[  56.08 ~  58.40 ]  그럼 도대체 일시 정지가 몇초냐는 건데요.
[  58.40 ~  61.76 ]  지금 인터넷에 보면 3초다 5초다 말들이 많

In [49]:
whisper_result

,start,end,text
0,0.00,4.00,자 오늘은 지금 전국민이 혼란에 빠진 우회전 관련 최종정리 영상인데요.
1,4.00,7.04,이거 뉴스에서도 정작 중요한 건 안 알려져서 거의 깔리게 만드는
2,7.04,8.60,시간엽수니 핵심한 빠르게가 부족.
3,8.60,12.68,일단 첫 번째 사실 지금 단속되고 있는 차량 90%는 다 이것 때문인데요.
4,12.68,15.48,바로 도대체 어디서 일시 정지를 해야 되냐는 겁니다.
5,15.48,21.08,결론부터 말씀드리면 대부분 여기 우회전에서 만나는 보행자 신호등에서 멈춰야 되는 ...
6,21.08,21.68,아닙니다.
7,21.68,26.44,정답은 바로 여기 우회전하기 전에 만나는 횡단보도에 있는 정지선에서 일시 정지해야...
8,26.44,28.64,확실하게 경찰청 공식 입장을 들어보죠.
9,28.64,33.44,앞에 운영이 진행하고자 하는 방향에 적식 적신호가 들어왔을 때는
